# JSON Schema generation

A simple usage of the library that, given generates a JSON Schema for inputs and outputs.

In [22]:
pip install cwl_loader jsonschema

Note: you may need to restart the kernel to use updated packages.


## 1. Parsing

In this sample we'll show the access from a remote public URL.

In [23]:
from cwl_loader import load_cwl_from_location
from cwl2ogc import BaseCWLtypes2OGCConverter

workflow_id = 'pattern-12'
cwl_document = load_cwl_from_location('https://raw.githubusercontent.com/eoap/application-package-patterns/refs/heads/main/cwl-workflow/pattern-12.cwl')

workflow = None
for wf in cwl_document:
    if workflow_id == wf.id.split('#')[-1]:
        workflow = wf
        break

if workflow is not None:
    cwl_converter = BaseCWLtypes2OGCConverter(workflow)
else:
    raise ValueError(f"'#{workflow_id}' not found in input $graph")

2025-09-08 11:27:55.919 | DEBUG    | cwl_loader:load_cwl_from_location:196 - Loading CWL document from https://raw.githubusercontent.com/eoap/application-package-patterns/refs/heads/main/cwl-workflow/pattern-12.cwl...
2025-09-08 11:27:56.266 | DEBUG    | cwl_loader:_load_cwl_from_stream:199 - Reading stream from https://raw.githubusercontent.com/eoap/application-package-patterns/refs/heads/main/cwl-workflow/pattern-12.cwl...
2025-09-08 11:27:56.280 | DEBUG    | cwl_loader:load_cwl_from_stream:173 - CWL data of type <class 'ruamel.yaml.comments.CommentedMap'> successfully loaded from stream
2025-09-08 11:27:56.281 | DEBUG    | cwl_loader:load_cwl_from_yaml:124 - Updating the model of type 'CommentedMap' to version 'v1.2'...
2025-09-08 11:27:56.281 | DEBUG    | cwl_loader:load_cwl_from_yaml:135 - Raw CWL document successfully updated to v1.2! Now converting to the CWL model...
2025-09-08 11:28:06.150 | DEBUG    | cwl_loader:load_cwl_from_yaml:143 - Raw CWL document successfully updated t

## 2. Inputs JSON Schema generation

Once the document is parsed, invoke the `cwl2ogc` APIs to convert the CWL inputs to the JSON schema:

In [24]:
import sys

cwl_converter.dump_inputs_json_schema(stream=sys.stdout, pretty_print=True)

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "https://eoap.github.io/cwl2ogc/pattern-12/inputs.yaml",
  "description": "The schema to represent a pattern-12 inputs definition",
  "type": "object",
  "required": [
    "aoi",
    "bands",
    "item",
    "cropped-collection",
    "ndwi-collection",
    "water-bodies-collection"
  ],
  "properties": {
    "aoi": {
      "$ref": "#/$defs/aoi"
    },
    "bands": {
      "$ref": "#/$defs/bands"
    },
    "item": {
      "$ref": "#/$defs/item"
    },
    "cropped-collection": {
      "$ref": "#/$defs/cropped-collection"
    },
    "ndwi-collection": {
      "$ref": "#/$defs/ndwi-collection"
    },
    "water-bodies-collection": {
      "$ref": "#/$defs/water-bodies-collection"
    }
  },
  "additionalProperties": false,
  "$defs": {
    "aoi": {
      "type": "object",
      "properties": {
        "bbox": {
          "type": "array",
          "items": {
            "type": "number",
            "format": "double

### 2.1 Inputs validation

Schema can be used to fully validate an inputs dictionary (expecting JSON Schema validation errors in the example below):

In [ ]:
from jsonschema import Draft202012Validator
from jsonschema.exceptions import SchemaError

def validate(schema: dict, data: dict):
    try:
        validator = Draft202012Validator(schema)
        errors = validator.iter_errors(data) if validator is not None else []

        if errors:
            for error in errors:
                print(f"[{'.'.join(error.schema_path)}] - #/{'/'.join(error.path)}: {error.message}")
        else:
            print('No JSON Schema violations detected!')
    except SchemaError as schema_error:
        print(f"An error occurred while instantiating {Draft202012Validator.__class__.__name__}: {schema_error.message}")


[required] - #/: 'cropped-collection' is a required property
[required] - #/: 'ndwi-collection' is a required property
[required] - #/: 'water-bodies-collection' is a required property
[properties.aoi.type] - #/aoi: '-118.985,38.432,-118.183,38.938' is not of type 'object'
[additionalProperties] - #/: Additional properties are not allowed ('filesB' was unexpected)


Define the inputs to be validate

In [ ]:
inputs = {
    "aoi": "-118.985,38.432,-118.183,38.938",
    "filesB": "EPSG:4326",
    "bands": [ "green", "nir08" ],
    "item": "https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/items/LC08_L2SP_042033_20231007_02_T1"
}

validate(cwl_converter.get_inputs_json_schema(), inputs)

## 3. Outputs JSON Schema generation

Users can reuse the `BaseCWLtypes2OGCConverter` instance to convert the CWL outputs to the JSON Schema:

In [35]:
cwl_converter.dump_outputs_json_schema(stream=sys.stdout, pretty_print=True)

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "https://eoap.github.io/cwl2ogc/pattern-12/outputs.yaml",
  "description": "The schema to represent a pattern-12 outputs definition",
  "type": "object",
  "required": [
    "cropped",
    "ndwi",
    "water_bodies"
  ],
  "properties": {
    "cropped": {
      "$ref": "#/$defs/cropped"
    },
    "ndwi": {
      "$ref": "#/$defs/ndwi"
    },
    "water_bodies": {
      "$ref": "#/$defs/water_bodies"
    }
  },
  "additionalProperties": false,
  "$defs": {
    "cropped": {
      "type": "array",
      "items": {
        "oneOf": [
          {
            "type": "string",
            "format": "uri"
          },
          {
            "$ref": "https://raw.githubusercontent.com/radiantearth/stac-api-spec/refs/tags/v1.0.0/stac-spec/item-spec/json-schema/item.json"
          }
        ]
      }
    },
    "ndwi": {
      "oneOf": [
        {
          "type": "string",
          "format": "uri"
        },
        {
 

### 2.1 Outputs validation

Schema can be used to fully validate an outputs dictionary (JSON Schema validation expected to pass):

In [ ]:
outputs = {
    "example_out": "In girum imus nocte et consumimur igni"
}

validate(cwl_converter.get_outputs_json_schema(), outputs)